In [2]:
import cv2
import face_recognition
import datetime

def largest_face_location(face_locations: list):
    if len(face_locations) != 0:
        largest_face = max(face_locations, key=lambda face: (face[2] - face[0]) * (face[3] - face[1])) # calculates the area of bounding box and selects max one

        return largest_face
    else:
        return None  # Return None if the list is empty

location, encoding, landmark, timing, countdown = None, None, None, None, None

video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()

    rgb_frame = frame[:,:,::-1]

    face_locations = face_recognition.api.face_locations(rgb_frame) # For face detection

    if len(face_locations) != 0:

        if location == None:
            location = [largest_face_location(face_locations)]
            encoding = face_recognition.api.face_encodings(rgb_frame, location)[0]
            landmark = face_recognition.api.face_landmarks(rgb_frame, location)[0]
            timing = datetime.datetime.now()
            countdown = 5

        else:
            face_encodings = face_recognition.api.face_encodings(rgb_frame, face_locations)
            comparison = face_recognition.api.compare_faces(face_encodings, encoding)
            if True not in comparison:
                location = [largest_face_location(face_locations)]
                encoding = face_recognition.api.face_encodings(rgb_frame, location)[0]
                landmark = face_recognition.api.face_landmarks(rgb_frame, location)[0]
                timing = datetime.datetime.now()
                countdown = 5
            else:
                index = comparison.index(True)
                location = [face_locations[index]]
                encoding = face_encodings[index]
                landmark = face_recognition.api.face_landmarks(rgb_frame, location)[0]
                current_time = datetime.datetime.now()
                if (current_time - timing).total_seconds() >= 1:
                    timing = current_time

                    if isinstance(countdown, int) and countdown > 1:
                        countdown = countdown - 1
                    else:
                        ######################
                        countdown = 'blah blah'
                        ######################

        (top,right,bottom,left) = location[0]
        cv2.rectangle(frame,(left,top),(right,bottom),(0,255,255),2)
        cv2.putText(frame, str(countdown), (right -130, top -50), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4)

    
    else:
        location, encoding, landmark, timing, countdown = None, None, None, None, None
    
    cv2.imshow('rgb_frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
